In [2]:
import boto3
import os
import time
import urllib
import json
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms.bedrock import Bedrock
from reportlab.pdfgen import canvas
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import S3FileLoader

transcribe_client = boto3.client('transcribe')
from bedrock import get_bedrock_client
bedrock_client = get_bedrock_client(region='us-east-1', runtime=True)

def transcribe_file(job_name, file_uri, transcribe_client):
    transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': file_uri},
        MediaFormat='mp4',
        LanguageCode='en-US'
    )
    max_tries = 60
    while max_tries > 0:
        max_tries -= 1
        job = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        job_status = job['TranscriptionJob']['TranscriptionJobStatus']
        if job_status in ['COMPLETED', 'FAILED']:
            print(f"Job {job_name} is {job_status}.")
            if job_status == 'COMPLETED':
                response = urllib.request.urlopen(job['TranscriptionJob']['Transcript']['TranscriptFileUri'])
                data = json.loads(response.read())
                text = data['results']['transcripts'][0]['transcript']
            break
        else:
            print(f"Waiting for {job_name}. Current status is {job_status}.")
        time.sleep(10)
    return text

file_uri = 's3://my-s3-doc-loader/aws1.mp4'
text = transcribe_file('My-video-to-text', file_uri, transcribe_client)

pdf_filename = 'Video_text_output.pdf'
pdf = canvas.Canvas(pdf_filename)

pdf.setFont("Helvetica", 12)
pdf.drawString(10, 800, "Transcribed Text:")
text_lines = text.split('\n')
for i, line in enumerate(text_lines):
    pdf.drawString(10, 780 - i * 15, line)

pdf.save()

s3_bucket_name = 'my-s3-doc-loader'
s3_key = pdf_filename
s3_client = boto3.client('s3')
with open(pdf_filename, 'rb') as pdf_file:
    s3_client.upload_fileobj(pdf_file, s3_bucket_name, s3_key)



Waiting for My-video-to-text. Current status is IN_PROGRESS.
Waiting for My-video-to-text. Current status is IN_PROGRESS.
Waiting for My-video-to-text. Current status is IN_PROGRESS.
Waiting for My-video-to-text. Current status is IN_PROGRESS.
Waiting for My-video-to-text. Current status is IN_PROGRESS.
Waiting for My-video-to-text. Current status is IN_PROGRESS.
Waiting for My-video-to-text. Current status is IN_PROGRESS.
Job My-video-to-text is COMPLETED.


In [3]:
loader = S3FileLoader(bucket=s3_bucket_name, key=pdf_filename)
file_content = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

c:\Users\Lenovo\anaconda3\Lib\site-packages\onnxruntime\capi\_pybind_state.py:26: UserWarning: Please install the 2019 Visual C++ runtime and then try again. If you've installed the runtime in a non-standard location (other than %SystemRoot%\System32), make sure it can be found by setting the correct path.
  warnings.warn("Please install the 2019 Visual C++ runtime and then try again. "


In [4]:
texts = text_splitter.split_documents(file_content)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(documents=texts, embedding=embeddings)
retriever = db.as_retriever(search_type='mmr', search_kwargs={"k": 3})
template = """
Human: Answer the question as truthfully as possible using only the provided text
text:{context}
question:{question}
Assistant:"""
qa_prompt = PromptTemplate(template=template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": qa_prompt}
llm = Bedrock(model_id="anthropic.claude-v2", client=bedrock_client)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=False
)
question = "why AWS?"
result = qa.run(question)
print(result)

 Based on the provided text, some key reasons to use AWS are:

- Pay-as-you-go pricing without upfront costs or long-term commitments. You only pay for the services you use.

- Access to a wide variety of cloud computing services that can be used to build and run virtually any type of application. 

- Access to the large AWS partner network of systems integrators, software vendors, and customers across industries. This provides expertise, integrations, and community support.

- Operational expertise and experience from AWS to support mission critical, risk sensitive organizations. 

- Ability to quickly scale, become more agile, and innovate faster using AWS services.

- More services and more features within those services compared to other cloud providers.

So in summary, the flexibility, breadth of services, ecosystem, expertise, and pay-as-you-go pricing make AWS an attractive cloud computing platform. The text highlights these as key reasons to choose AWS.
